In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tqdm.notebook as tqdm
import copy
import os

from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.optim.lr_scheduler import StepLR


import sys
sys.path.append('/Users/Matt/projects/entrovae/')
sys.path.append('/Users/Matt/projects/sgpvae/')

import sgpvae
import entrovae

## Data preparation.

In [2]:
transform=transforms.Compose([
        transforms.ToTensor(),
        ])

train_dataset = datasets.MNIST('../data', train=True, download=True,
                               transform=transform)
test_dataset = datasets.MNIST('../data', train=False,
                              transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

## Classifier training.

In [3]:
def train_cls(model, train_loader, optimiser, epoch):
    model.train()
    batch_iter = tqdm.tqdm(enumerate(train_loader), desc='Batch')
    for batch_idx, (x, y) in batch_iter:
        optimiser.zero_grad()
        loss, _ = model.nll(x, y)
        loss.backward()
        optimiser.step()
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test_cls(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            loss, output = model.nll(x, y)
            test_loss += loss.item()
            
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(y.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [4]:
cls = entrovae.classifiers.MNISTClassificationNet()
cls_optimiser = optim.Adadelta(cls.parameters(), lr=1.0)
cls_scheduler = StepLR(cls_optimiser, step_size=1, gamma=0.7)

In [5]:
for epoch in range(1, 14+1):
    train_cls(cls, train_loader, cls_optimiser, epoch)
    test_cls(cls, test_loader)
    cls_scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.297461
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.640176
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.142019
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.606228
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.444304
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.304913
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.412512
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.304198
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.460390
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.145553
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.535399
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.194456
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.274349
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.267375
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.268513
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.329536
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.134339
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.361926
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.315745
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.179050
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.070912
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.034499
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.063857
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.071959
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.059939
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.087352
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.022285
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.046785
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.026579
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.098079
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.025128
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.060161
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.015940
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.056434
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.214289
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.081840
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.127615
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.053281
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.138476
Train Epoch: 2 [12160/60000 (20%)]	Loss: 0.071659
Train Epoch: 2 [12800/60000 (

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.049496
Train Epoch: 3 [640/60000 (1%)]	Loss: 0.028434
Train Epoch: 3 [1280/60000 (2%)]	Loss: 0.018326
Train Epoch: 3 [1920/60000 (3%)]	Loss: 0.034531
Train Epoch: 3 [2560/60000 (4%)]	Loss: 0.018847
Train Epoch: 3 [3200/60000 (5%)]	Loss: 0.029280
Train Epoch: 3 [3840/60000 (6%)]	Loss: 0.070324
Train Epoch: 3 [4480/60000 (7%)]	Loss: 0.010634
Train Epoch: 3 [5120/60000 (9%)]	Loss: 0.046441
Train Epoch: 3 [5760/60000 (10%)]	Loss: 0.007166
Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.006881
Train Epoch: 3 [7040/60000 (12%)]	Loss: 0.084415
Train Epoch: 3 [7680/60000 (13%)]	Loss: 0.069326
Train Epoch: 3 [8320/60000 (14%)]	Loss: 0.033645
Train Epoch: 3 [8960/60000 (15%)]	Loss: 0.045498
Train Epoch: 3 [9600/60000 (16%)]	Loss: 0.064082
Train Epoch: 3 [10240/60000 (17%)]	Loss: 0.003642
Train Epoch: 3 [10880/60000 (18%)]	Loss: 0.027076
Train Epoch: 3 [11520/60000 (19%)]	Loss: 0.002784
Train Epoch: 3 [12160/60000 (20%)]	Loss: 0.030280
Train Epoch: 3 [12800/60000 (

KeyboardInterrupt: 

## Set up EntroVAE and VAE datasets. 
* For ***EntroVAE*** use entropies as well as the data.
* For ***VAE***, dataset is simply the images.

In [6]:
pred_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)
probs = torch.zeros(len(train_dataset), 10)
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(pred_loader):
        probs[batch_idx*64:(batch_idx+1)*64] = cls(data).detach().exp()
        
entropies = - (probs * probs.log()).sum(1)

In [7]:
class EntroVAEDataset(torch.utils.data.Dataset):
    def __init__(self, data, entropy):
        self.data = data
        self.entropy = entropy
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        return self.data[idx, ...], self.entropy[idx]
    
entrovae_dataset = EntroVAEDataset(train_dataset.data, entropies)
entrovae_loader = torch.utils.data.DataLoader(entrovae_dataset, batch_size=64)
vae_loader = torch.utils.data.DataLoader(train_dataset.data, batch_size=64)

In [8]:
def train_vae(model, train_loader, optimiser, epoch):
    model.train()
    train_loss = 0
    batch_iter = tqdm.tqdm(enumerate(train_loader), desc='Batch')
    
    for batch_idx, x in batch_iter:
        optimiser.zero_grad()
        
        if isinstance(x, tuple) or isinstance(x, list):
            x, h = x
            x = x.view(-1, 784).float() / 255
            loss = -model.elbo(x, h)
        else:
            x = x.view(-1, 784).float() / 255
            loss = -model.elbo(x)
            
        loss.backward()
        train_loss += loss.item()
        optimiser.step()
        
        if batch_idx % 10 == 0:
            batch_iter.set_postfix(loss=loss.item())
            
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [9]:
z_dim = 2
x_dim = 784

encoder = sgpvae.networks.LinearGaussian(x_dim, z_dim, [512, 256], min_sigma=1e-3)
loglikelihood = entrovae.loglikelihoods.NNBernoulli(z_dim, x_dim, [256, 512])

vae_model = entrovae.models.VAE(copy.deepcopy(encoder), copy.deepcopy(loglikelihood), z_dim)
entrovae_model = entrovae.models.EntroVAE(copy.deepcopy(encoder), copy.deepcopy(loglikelihood), z_dim)

## VAE training.

In [10]:
vae_optimiser = optim.Adam(vae_model.parameters())
for epoch in range(1, 14+1):
    train_vae(vae_model, vae_loader, vae_optimiser, epoch)


====> Epoch: 1 Average loss: 2.8252



====> Epoch: 2 Average loss: 2.4959



====> Epoch: 3 Average loss: 2.3994



====> Epoch: 4 Average loss: 2.3392



====> Epoch: 5 Average loss: 2.2988



====> Epoch: 6 Average loss: 2.2699



====> Epoch: 7 Average loss: 2.2503



====> Epoch: 8 Average loss: 2.2311



====> Epoch: 9 Average loss: 2.2147


KeyboardInterrupt: 

In [11]:
with torch.no_grad():
    z = torch.zeros(100, 2)
    for i, x in enumerate(torch.linspace(-2, 2, 10, dtype=torch.float32)):
        for j, y in enumerate(torch.linspace(-2, 2, 10, dtype=torch.float32)):
            idx = 10 * i + j
            z[idx, :] = torch.tensor([x, y])
            
    sample = vae_model.loglikelihood.predict(z)
    
    filename = './samples/vae_sample'
    
    if os.path.exists(filename + '.png'):
        i = 1
        while os.path.exists(filename + '_' + str(i) + '.png'):
            i += 1
            
        filename = filename + '_' + str(i) + '.png'
        
    else:
        filename = filename + '.png'
    
    save_image(sample.view(100, 1, 28, 28), filename)

## EntroVAE training.

In [12]:
entrovae_optimiser = optim.Adam(entrovae_model.parameters())
for epoch in range(1, 14+1):
    train_vae(entrovae_model, entrovae_loader, entrovae_optimiser, epoch)


====> Epoch: 1 Average loss: 2.8848



====> Epoch: 2 Average loss: 2.5583



====> Epoch: 3 Average loss: 2.4653



====> Epoch: 4 Average loss: 2.4169



====> Epoch: 5 Average loss: 2.3779


...


====> Epoch: 8 Average loss: 2.3121



====> Epoch: 9 Average loss: 2.2967



====> Epoch: 10 Average loss: 2.2898



====> Epoch: 11 Average loss: 2.2825


KeyboardInterrupt: 

In [14]:
with torch.no_grad():
    z = torch.zeros(100, 2)
    for i, x in enumerate(torch.linspace(-2, 2, 10, dtype=torch.float32)):
        for j, y in enumerate(torch.linspace(-2, 2, 10, dtype=torch.float32)):
            idx = 10 * i + j
            z[idx, :] = torch.tensor([x, y])
            
    sample = entrovae_model.loglikelihood.predict(z)
    
    filename = './samples/entrovae_sample'
    
    if os.path.exists(filename + '.png'):
        i = 1
        while os.path.exists(filename + '_' + str(i) + '.png'):
            i += 1
            
        filename = filename + '_' + str(i) + '.png'
        
    else:
        filename = filename + '.png'
    
    save_image(sample.view(100, 1, 28, 28), filename)

## Compare disentalglement of latent space?
* Not sure what to use here tbh.